<a href="https://colab.research.google.com/github/PioneerAlexander/Leveraging-software-evolution-data-with-LLMs/blob/main/Refact-1_6B_eval_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [106]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [107]:
!pip install datasets

In [108]:
from datasets import load_dataset
ds = load_dataset("bigcode/humanevalpack", "python")['test']

In [109]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "smallcloudai/Refact-1_6B-fim"
device = "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

In [119]:
prompt_template = "<empty_output>SYSTEM {system}\n" \
                  "<empty_output>USER {query}\n" \
                  "<empty_output>ASSISTANT\n"

In [156]:
def generate_prompt(system, task, with_tests=False):
    query_template="Question: Fix the bugs in {entry_point}: \n {import_desc} \ndef {signature}: " \
    "\n{buggy_solution} {test}\nAnswer: \n {import_desc} \ndef {signature}: \n"
    return prompt_template.format(system=system,
                                  query=query_template.format(entry_point=task['entry_point'],
                                                              import_desc=task['import'],
                                                              signature=task['signature'],
                                                              buggy_solution=task['buggy_solution'],
                                                              test=task['test'] if with_tests else ""))

In [166]:
import re

def get_model_answer(text: str) -> str:
  pattern = r'<empty_output>ASSISTANT\n(.*?)<empty_output>'

  match = re.search(pattern, tokenizer.decode(outputs[0]), re.DOTALL)

  return match.group(1) if match else ""

In [1]:
import random

def calculate_zero_shot_pass_metric(ds):

    total_number = 20
    correct_number = 0

    ds_sample = random.choices(ds, k=total_number)

    print([task['task_id'] for task in ds_sample])

    for task_number, task in enumerate(ds_sample):
      inputs = tokenizer.encode(generate_prompt("", task, with_tests=True), return_tensors="pt").to(device)
      outputs = model.generate(inputs, max_length=10000, temperature=0.2, pad_token_id=tokenizer.eos_token_id)
      code_with_tests = "from typing import List\n" + get_model_answer(tokenizer.decode(outputs[0])) + task['test']
      try:
        exec(code_with_tests)
        correct += 1
      except:
        pass
      print("The iteration number: {number}. Correct answers: {correct_number}/{total_number}\n".format(number=task_number,
                                                                                                        correct_number=correct_number,
                                                                                                        total_number=total_number))

    return correct_number / total_number

print(calculate_zero_shot_pass_metric(ds))

NameError: ignored

In [182]:
inputs = tokenizer.encode(generate_prompt("", ds[130], with_tests=True), return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=10000, temperature=0.2, pad_token_id=tokenizer.eos_token_id)
print("-"*80)
print(tokenizer.decode(outputs[0]))
code_with_tests = "from typing import List\n" + get_model_answer(tokenizer.decode(outputs[0])) + ds[130]['test']
exec(code_with_tests)
try:
    exec(code_with_tests)
    correct += 1
except:
    pass
print(correct)

--------------------------------------------------------------------------------
<empty_output>SYSTEM 
<empty_output>USER Question: Fix the bugs in tri: 
  
def tri(n): 
    if n == 0:
        return [1]
    my_tri = [1, 3]
    for i in range(2, n + 1):
        if i % 2 == 0:
            my_tri.append(i / 2 + 1)
        else:
            my_tri.append(my_tri[i - 1] + my_tri[i - 2] + i + (i + 3) / 2)
    return my_tri
 def check(tri):

    # Check some simple cases
    
    assert tri(3) == [1, 3, 2.0, 8.0]
    assert tri(4) == [1, 3, 2.0, 8.0, 3.0]
    assert tri(5) == [1, 3, 2.0, 8.0, 3.0, 15.0]
    assert tri(6) == [1, 3, 2.0, 8.0, 3.0, 15.0, 4.0]
    assert tri(7) == [1, 3, 2.0, 8.0, 3.0, 15.0, 4.0, 24.0]
    assert tri(8) == [1, 3, 2.0, 8.0, 3.0, 15.0, 4.0, 24.0, 5.0]
    assert tri(9) == [1, 3, 2.0, 8.0, 3.0, 15.0, 4.0, 24.0, 5.0, 35.0]
    assert tri(20) == [1, 3, 2.0, 8.0, 3.0, 15.0, 4.0, 24.0, 5.0, 35.0, 6.0, 48.0, 7.0, 63.0, 8.0, 80.0, 9.0, 99.0, 10.0, 120.0, 11.0]

    # Chec

AssertionError: ignored

In [129]:
print(ds[2]['canonical_solution'])

    return number % 1.0



In [162]:

correct = 0

exec("from typing import List\n" + result + ds[130]['test'])
try:
  exec("from typing import List\n" + result + ds[0]['test'])
  correct += 1
except:
  pass
print(correct)

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx!= idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False
 




def check(has_close_elements):
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
    assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
    assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False

check(has_close_elements)

1
